<!-- サンチャゴの大暴落で勝つから引用 -->
<!-- 移動平均線からの乖離が大きい銘柄に対してエントリー -->

In [10]:
import pandas as pd
import numpy as np
import os
import sys
sys.path.append("../")
from common.evaluate.evaluate import evaluation_index
import codecs
from joblib import Parallel, delayed

In [11]:
code_name = 'トヨタ自'
df = pd.read_csv(f'../data/candle_data/TOPIX/{code_name}.csv',index_col = 0)
display(df)

,high,low,open,cl,volume,close,price,sma5,sma_close_ratio,sma_close_ratio_5,sma25,sma_close_ratio_25,sma45,sma_close_ratio_45,sma75,sma_close_ratio_75
Date,,,,,,,,,,,,,,,,
2019-01-04,1272.000000,1232.199951,1236.000000,1269.199951,50059500.0,1160.423950,1196.995972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-07,1324.400024,1295.599976,1299.000000,1309.199951,38386000.0,1196.995972,1211.624634,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-08,1337.400024,1316.800049,1322.000000,1325.199951,39896500.0,1211.624634,1220.950317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-09,1343.000000,1325.800049,1342.400024,1335.400024,27472000.0,1220.950317,1225.887695,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-10,1342.800049,1325.400024,1326.000000,1340.800049,31702000.0,1225.887695,1247.282227,1203.176514,1.887602,1.887602,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-09,2282.500000,2244.500000,2269.000000,2245.000000,22370500.0,2245.000000,2178.500000,2230.000000,0.672646,0.672646,2187.24,2.640771,2103.109956,6.746677,2147.632539,4.533711
2022-05-10,2204.000000,2155.000000,2190.000000,2178.500000,28054600.0,2178.500000,2082.000000,2232.600000,-2.423184,-2.423184,2185.64,-0.326678,2104.708062,3.506041,2145.207171,1.551963
2022-05-11,2210.000000,2050.500000,2205.500000,2082.000000,64821200.0,2082.000000,2050.000000,2201.900000,-5.445297,-5.445297,2180.02,-4.496289,2106.235718,-1.150665,2141.060586,-2.758473


In [12]:
def func(file,path):
    # try:
    data = pd.read_csv(path+'/'+file,index_col=0)
    data = data.rename(columns={'Open': 'open', 'High': 'high','Low': 'low','Close': 'cl','Adj Close':'close','Volume': 'volume'})

    data['price'] = data['close'].shift(-1) 
    for sma in [5,25,45,75]:
        data[f"sma{sma}"] = data['close'].rolling(sma).mean()
        data[f'sma_close_ratio_{sma}'] = (data['close'] / data[f"sma{sma}"] - 1) * 100
    data.to_csv(path+'/'+file)
    print(data)
    
    # except:
    #     print(file)
    #     code = file

path = "../data/candle_data/TOPIX"

files = os.listdir(path)
Parallel(n_jobs=-1)(delayed(func)(file,path) for file in files)

              high     low  ...        sma75  sma_close_ratio_75
Date                        ...                                 
2019-01-04  3270.0  3170.0  ...          NaN                 NaN
2019-01-07  3360.0  3185.0  ...          NaN                 NaN
2019-01-08  3230.0  3150.0  ...          NaN                 NaN
2019-01-09  3250.0  3200.0  ...          NaN                 NaN
2019-01-10  3245.0  3160.0  ...          NaN                 NaN
...            ...     ...  ...          ...                 ...
2022-05-09  4085.0  3875.0  ...  3984.133333           -2.739199
2022-05-10  3860.0  3790.0  ...  3984.066667           -3.741571
2022-05-11  3915.0  3790.0  ...  3984.000000           -4.367470
2022-05-12  3930.0  3750.0  ...  3982.000000           -5.072828
2022-05-13  4065.0  3780.0  ...  3983.600000            1.792349

[814 rows x 16 columns]
              high     low  ...       sma75  sma_close_ratio_75
Date                        ...                                
20

EmptyDataError: No columns to parse from file

In [ ]:
def func(file,date,path):
    try:
        data = pd.read_csv(path+'/'+file,index_col=0)
        sma_close_ratio = data.loc[date,'sma_close_ratio']
        price = data.loc[date,'price']
        code = file
    except:
        sma_close_ratio = 0
        price = 0
        code = file

    return sma_close_ratio, price, code



: 

In [ ]:
def calcurate_prediction(date, sma):
    path = "../data/candle_data/TOPIX"
   
    files = os.listdir(path)
    res = Parallel(n_jobs=-1)(delayed(func)(file, date,path) for file in files[:500])
    result = pd.DataFrame(res,
                  columns=['sma_close_ratio', 'price', 'code'])

    result = result.sort_values('sma_close_ratio',ascending=True)

    return result.iloc[0,:]


: 

In [ ]:
def backtest(df,exit_range,sma,entry_threshold):
    code = ''
    df = df.rename(columns={'Open': 'open', 'High': 'high','Low': 'low','Close': 'close','Volume': 'volume'})
    n = df['close'].values.size
    y = df['close'].values.copy() * 0.0
    poss = df['close'].values.copy() * 0.0
    ret = 0.0
    pos = 0.0
    buy_entry_price = 1.0
    sell_entry_price = 1.0
    buy_entry_prices = np.full((n), np.nan)
    sell_entry_prices = np.full((n), np.nan)
    buy_exit_prices = np.full((n), np.nan)
    sell_exit_prices = np.full((n), np.nan)
    for i in range(len(df)):
        prev_pos = pos
        date = df.index[i]

        # Exit of long
        if prev_pos == 1 and (i - entry_date) >= exit_range:
            path = "../data/candle_data/TOPIX"
            data = pd.read_csv(path+'/'+code,index_col=0)
            data = data.rename(columns={'Open': 'open', 'High': 'high','Low': 'low','Close': 'close','Volume': 'volume'})

            data['price'] = data['close'].shift(-1) 
            sell_exit_price = data.loc[date,'price']
            sell_exit_prices[i] = data.loc[date,'price']
            ret += (sell_exit_price / buy_entry_price - 1)
            pos -= 1

        # entry
        if prev_pos == 0:
            result = calcurate_prediction(date, sma)
            if result[f'sma_close_ratio_{sma}'] <= entry_threshold:
                buy_entry_price = result['price']
                buy_entry_prices[i] = result['price']
                code = result['code']
                entry_date = i
                pos += 1

        y[i] = ret
        poss[i] = pos

    return y, poss, buy_entry_prices, sell_entry_prices, buy_exit_prices, sell_exit_prices


exit_range = 5
sma = 25
entry_threshold = -20

for exit_range in [1,3,5]:
    for sma in [5,25,45,75]:
        for entry_threshold in [-5,-10,-20,-30]:
            print('==================')
            print(exit_range, sma, entry_threshold)

            cumulative_return, possition, buy_entry_prices, sell_entry_prices, buy_exit_prices, sell_exit_prices = backtest(df,exit_range,sma,entry_threshold)
            df_result =  pd.DataFrame(
                {
                    "cumulative_return": cumulative_return,
                    "position": possition,
                    "buy_entry_price": buy_entry_prices,
                    "sell_entry_price": sell_entry_prices,
                    "buy_exit_price": buy_exit_prices,
                    "sell_exit_price": sell_exit_prices,
                },
                index=df.index
            )
            result_dict = evaluation_index(df_result)


df_result['cumulative_return'].plot()
plt.title('cumulative_return')
plt.show()

print('ポジション推移です。変動が細かすぎて青色一色になっていると思います。')
print('ちゃんと全ての期間でトレードが発生しているので、正常です。')
df_result['position'].plot()
plt.title('position')
plt.show()

print('ポジションの平均の推移です。どちらかに偏りすぎていないかなどを確認できます。')
df_result['position'].rolling(1000).mean().plot()
plt.title('position_average')
plt.show()

print('取引量(ポジション差分の絶対値)の累積です。')
print('期間によらず傾きがだいたい同じなので、全ての期間でちゃんとトレードが行われていることがわかります。')
df_result['position'].diff(1).abs().dropna().cumsum().plot()
plt.title('cumulative_position')

1 5 -5


KeyboardInterrupt: 

: 